# Demo Notebook how to run models on static mouse datasets

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2 

In [2]:
import os
import datajoint as dj
dj.config['database.host'] = os.environ['DJ_HOST']
dj.config['database.user'] = os.environ['DJ_USER']
dj.config['database.password'] = os.environ['DJ_PASS']
dj.config['enable_python_native_blobs'] = True
dj.config['display.limit'] = 200

name = 'iclr' #"test"
dj.config['schema_name'] = f"konstantin_nnsysident_{name}"

In [3]:
import torch
import numpy as np
import pickle 
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 10)
from collections import OrderedDict, Iterable
import matplotlib.pyplot as plt
import matplotlib
import re
import seaborn as sns

import nnfabrik
from nnfabrik.main import *
from nnfabrik import builder
from nnfabrik.utility.hypersearch import Bayesian
from dataport.bcm.static import PreprocessedMouseData

from nnsysident.tables.experiments import *
from nnsysident.tables.bayesian import *
from nnsysident.datasets.mouse_loaders import static_shared_loaders
from nnsysident.datasets.mouse_loaders import static_loaders
from nnsysident.datasets.mouse_loaders import static_loader
from nnsysident.tables.scoring import OracleScore, OracleScoreTransfer

<ipython-input-3-05b2d793f8c5>:7: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import OrderedDict, Iterable


Connecting konstantin@134.2.168.16:3306
Schema name: konstantin_nnsysident_iclr


---

In [4]:
# change path here
paths = ['data/static22564-2-12-preproc0.zip']

#paths = ['data/static22564-2-12-preproc0.zip']
dataset_fn = 'nnsysident.datasets.mouse_loaders.static_loaders'
dataset_config = dict(
    paths=paths,
    batch_size=64,
    seed=1,
    file_tree=True,
    #image_n=50,
    #image_base_seed=1,
    #multi_match_n=972,
    #multi_match_base_seed=1,
    #exclude_multi_match_n = 3625,

)

dataloaders = builder.get_data(dataset_fn, dataset_config)

data/static22564-2-12-preproc0 exists already. Not unpacking data/static22564-2-12-preproc0.zip


# Get Model

### The New gaussian readout: change gauss_type for the different modes

In [7]:
model_fn = 'nnsysident.models.models.se2d_fullgaussian2d'

model_config = {'init_mu_range': 0.55,
                 'init_sigma': 0.4,
                 'input_kern': 15,
                 'hidden_kern': 13,
                 'gamma_input': 1.0,
                 'grid_mean_predictor': None,
                 'gamma_readout': 2.439,
                 'share_features': False,
                 'share_transform': False}


model = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1)

# Get Trainer

In [8]:
trainer_fn = 'nnsysident.training.trainers.standard_trainer'
trainer_config = dict(track_training=True) #detach_core=True, 
trainer = builder.get_trainer(trainer_fn, trainer_config)

# Run Training

In [9]:
score, output, model_state = trainer(model=model, dataloaders=dataloaders, seed=1)

correlation 0.0005847462
poisson_loss 5093235.0


Epoch 1:  34%|███▍      | 24/70 [00:02<00:04,  9.53it/s]


KeyboardInterrupt: 

In [ ]:
np.unique(dataloaders['train']['24391-19-14-0'].dataset.neurons.layer)

In [ ]:
grid = model.readout['24391-19-14-0'].grid.squeeze().cpu().data.numpy()

In [ ]:
grid

In [ ]:
plt.scatter(*grid.T)
plt.xlim(-1,1)
plt.ylim(-1,1)
plt.savefig('zhiwei_mus.png')

In [ ]:
features = model.readout['24391-19-14-0'].features.cpu().data.numpy().squeeze()

In [ ]:
plt.hist(features.flatten(), bins=50)
plt.savefig('zhiwei_features.png')

In [ ]:
torch.save(grid, 'zhiwei_mus')

In [ ]:
grid.T.shape

In [ ]:
grid.shape